# Meta - Photo Sharing Platform User Engagement Metrics

```SQL

CREATE TABLE dim_user (
    user_id INTEGER,
    age INTEGER,
    country VARCHAR
);

CREATE TABLE fct_photo_sharing (
    photo_id INTEGER,
    user_id INTEGER,
    shared_date DATE
);

INSERT INTO dim_user (user_id, age, country)
VALUES
    (1, 16, 'Canada'),
    (2, 55, 'Mexico'),
    (3, 30, 'United States'),
    (4, 17, 'United States'),
    (5, 60, 'United States'),
    (6, 22, 'France'),
    (7, 15, 'Germany'),
    (8, 51, 'Brazil'),
    (9, 40, 'India'),
    (10, 65, 'United States'),
    (11, 14, 'United Kingdom'),
    (12, 33, 'United States'),
    (13, 70, 'Japan'),
    (14, 18, 'Australia'),
    (15, 50, 'United States');

INSERT INTO fct_photo_sharing (photo_id, user_id, shared_date)
VALUES
    (1, 1, '2024-07-05'),
    (2, 2, '2024-07-10'),
    (3, 3, '2024-07-15'),
    (4, 4, '2024-07-20'),
    (5, 5, '2024-07-25'),
    (6, 7, '2024-07-12'),
    (7, 8, '2024-07-28'),
    (8, 10, '2024-07-30'),
    (9, 1, '2024-08-05'),
    (10, 2, '2024-08-15'),
    (11, 6, '2024-08-20'),
    (12, 7, '2024-08-10'),
    (13, 8, '2024-08-25'),
    (14, 11, '2024-08-28'),
    (15, 1, '2024-09-03'),
    (16, 2, '2024-09-07'),
    (17, 8, '2024-09-15'),
    (18, 11, '2024-09-20'),
    (19, 13, '2024-09-25'),
    (20, 4, '2024-09-10');

SELECT * FROM dim_user;

SELECT * FROM fct_photo_sharing;
```

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_user = pd.read_csv('Data/015/dim_user.csv')
df_photo = pd.read_csv('Data/015/fct_photo_sharing.csv')

df_user.head()

,user_id,age,country
0,1,16,Canada
1,2,55,Mexico
2,3,30,United States
3,4,17,United States
4,5,60,United States


In [3]:
df_photo.head()

,photo_id,user_id,shared_date
0,1,1,2024-07-05
1,2,2,2024-07-10
2,3,3,2024-07-15
3,4,4,2024-07-20
4,5,5,2024-07-25


# Pregunta 1

### ¿Cuántas fotos fueron compartidas por usuarios que tienen menos de 18 años o más de 50 años durante julio de 2024? Esta métrica nos ayudará a entender si estos segmentos de edad están interactuando con la función de compartir fotos.

In [4]:
df_merge = pd.merge(df_user, df_photo, on='user_id')

df_julio = df_merge[
    (df_merge['shared_date'].between('2024-07-01','2024-07-31')) &
    ((df_merge['age'] > 50) | (df_merge['age'] < 18))
]

respuesta1 = df_julio['photo_id'].count()

respuesta1

np.int64(7)

```SQL
SELECT
    count(p.photo_id) AS total_photos
FROM fct_photo_sharing p
JOIN dim_user u ON p.user_id = u.user_id
WHERE p.shared_date BETWEEN '2024-07-01' AND '2024-07-31'
AND (u.age < 18 OR u.age > 50);
```

# Pregunta 2

### ¿Cuáles son los IDs de usuario y el número total de fotos compartidas por usuarios que no son de Estados Unidos durante agosto de 2024? Este análisis nos ayudará a identificar patrones de compromiso entre los usuarios internacionales.

In [5]:
df_merge = df_user.merge(df_photo, on='user_id')

df_agosto = df_merge[
    (df_merge['country'] != 'United States') &
    (df_merge['shared_date'].between('2024-08-01','2024-08-31'))
]

respuesta2 = df_agosto.groupby('user_id')['photo_id'].count().reset_index(name='total_photos')

respuesta2

,user_id,total_photos
0,1,1
1,2,1
2,6,1
3,7,1
4,8,1
5,11,1


```SQL
SELECT
    u.user_id,
    COUNT(p.photo_id) as total_photos
FROM dim_user u
JOIN fct_photo_sharing p ON p.user_id = u.user_id
WHERE p.shared_date BETWEEN '2024-08-01' AND '2024-08-31'
AND country <> 'United States'
GROUP BY u.user_id ;
```

# Pregunta 3

### ¿Cuál es el número total de fotos compartidas por usuarios que tienen menos de 18 años o más de 50 años y que no son de Estados Unidos durante el tercer trimestre de 2024? Esta medida nos informará si existen diferencias significativas en el uso entre estos segmentos de edad y geográficos.

In [7]:
df_merge = df_user.merge(df_photo, on='user_id')

df_q3 = df_merge[
    (df_merge['shared_date'].between('2024-07-01','2024-09-30')) &
    (df_merge['country'] != 'United States') &
    ((df_merge['age'] < 18) | (df_merge['age'] > 50))
]

respuesta3 = df_q3['photo_id'].count()

respuesta3

np.int64(14)

```SQL
SELECT
    count(p.photo_id) AS total_photos
FROM fct_photo_sharing p
JOIN dim_user u ON u.user_id = p.user_id
WHERE p.shared_date BETWEEN '2024-07-01' AND '2024-09-30'
AND u.country <> 'United States'
AND (u.age < 18 OR u.age > 50);
```